In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
X, y = load_iris(return_X_y=True)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,y,train_size=0.8)

#**ID3**

In [ ]:
import random

weather=list(range(0,4)) # rainy, sunny, cloudy, pleasent
temperature=list(range(0,3)) # High, medium, low
humidity=list(range(0,2)) # Normal, abnormal
target=[0,1]

x=[];y=[]

for i in range(1000):
  f1=random.choice(weather)
  f2=random.choice(temperature)
  f3=random.choice(humidity)
  f4=random.choice(target)
  x.append([f1,f2,f3])
  y.append(f4)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=)

In [ ]:
model_ID3=tree(criterion='entropy')
model_ID3.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [ ]:
print(accuracy_score(model_ID3.predict(x_test),y_test))

0.52


#**CART**

In [ ]:
model_CART=tree(criterion='gini')


In [ ]:
model_CART.fit(X_train,Y_train)

DecisionTreeClassifier()

In [ ]:
model_CART.score(X_test,Y_test)
print(model_CART.score(X_test,Y_test))
print(accuracy_score(model_CART.predict(X_test),Y_test))

1.0
1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**C4.5 from skratch**

In [ ]:
import pandas as pd
import numpy as np
import math
from collections import Counter

In [ ]:
path='/content/drive/MyDrive/Temp/PlayTennis.csv'
df=pd.read_csv(path)

In [ ]:
#Getting different uniques values that an attribute can take
s1=set(df['Outlook']);l1=len(s1)
s2=set(df['Temperature']);l2=len(s2)
s3=set(df['Humidity']);l3=len(s3)
s4=set(df['Wind']);l4=len(s4)
dictionary={'Outlook':s1,'Temperature':s2,'Humidity':s3,'Wind':s4}

In [ ]:
df    #Our training data

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [ ]:
def Partition_Attr(data):
# Some functions that are used in it
  def calculate_entropy(data_node,attr):
    values=set(data_node[attr])
    total_data=len(data_node)
    l={};l2=[]
    D=len(data_node)
    Entropy_of_parent=entropy_of_node(data_node)
    split_info=0
    for i in values:
      d=data_node.loc[data_node[attr]==i]
      classes=Counter(list(d['Play Tennis']))

      total_i=len(d);    # total_i = D_i
      E=0
      for c in classes:
        p=classes[c]/total_i
        E-=p*(math.log(p)/math.log(2))

      split_info+=(total_i/D)*(math.log(total_i/D)/math.log(2))
      l[i]=E
      l2.append((total_i/total_data)*E)
    return (Entropy_of_parent-sum(l2))/split_info

  attr=list(data.columns)
  print('All atributes',attr)
  Entropies=[]

  for i in attr[0:len(attr)-1]:
    Entropies.append(calculate_entropy(data,i))

# Return in format: ( attribute with minimum entropy and its entropy )
  return attr[np.argmin(np.array(Entropies))],min(Entropies)


In [ ]:
def entropy_of_node(data):
  total=len(data)
  d=Counter(data[data.columns[-1]])
  E=0
  for i in d:
    p=d[i]/total
    E-=(p)*math.log(p)/math.log(2)
  return E

In [ ]:
def Get_max_class(data):
  ''' It is assummed that the last column is of class'''
  pass
  v=list(data.columns)[-1]
  d=Counter(list(data[v]))
  m=max(d.values())
  for i in d:
    if d[i]==m:
      return i

In [ ]:
def train(data):
  print('*****************************************\n',data)

  # If data is pure: no partition
  if entropy_of_node(data) <=0.1 or len(data)<=3:
    return Get_max_class(data)

  #else: following part
  else:
      d=[]
      attr,E=Partition_Attr(data)
      for i in set(data[attr]):
        print(attr,'=',i)
        temp_list=train(data.loc[data[attr]==i].drop([attr],axis=1))
        d.append([attr,i,temp_list])
      return d


In [ ]:
model_C45=train(df)

*****************************************
      Outlook Temperature Humidity    Wind Play Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No
All atributes ['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Tennis']
Outlook = Rain
*****************************************
    Temperatur

In [ ]:
print('Trained model:',model_C45)
#Prediction function
def predict(model,input_data):
  for i in model:
    if input_data.loc[i[0]]==i[1]:
      if isinstance(i[2],list):
        ans=predict(i[2],input_data)
      else:
        return i[2]
  return ans

temp={'Predicted':[],'Original':[]}
for i in range(len(df)):
  x=df.loc[i]
  temp['Predicted'].append(predict(model_C45,x))
  temp['Original'].append(x['Play Tennis'])

df_accuracy=pd.DataFrame(temp)
df_accuracy

Trained model: [['Outlook', 'Rain', [['Wind', 'Weak', 'Yes'], ['Wind', 'Strong', 'No']]], ['Outlook', 'Sunny', [['Humidity', 'High', 'No'], ['Humidity', 'Normal', 'Yes']]], ['Outlook', 'Overcast', 'Yes']]


,Predicted,Original
0,No,No
1,No,No
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,No,No
6,Yes,Yes
7,No,No
8,Yes,Yes
9,Yes,Yes


#**ID3 for scratch**

In [ ]:
def Partition_Attr_ID3(data):
# Some functions that are used in it
  def calculate_entropy(data_node,attr):
    values=set(data_node[attr])
    total_data=len(data_node)
    l={};l2=[]
    D=len(data_node)
    Entropy_of_parent=entropy_of_node(data_node)
    for i in values:
      d=data_node.loc[data_node[attr]==i]
      classes=Counter(list(d['Play Tennis']))

      total_i=len(d);E=0    # total_i = D_i

      for c in classes:
        p=classes[c]/total_i
        E-=p*(math.log(p)/math.log(2))

      l[i]=E

      l2.append((total_i/total_data)*E)
    return Entropy_of_parent-sum(l2)

  attr=list(data.columns)
  print('All atributes',attr)
  Entropies=[]

  for i in attr[0:len(attr)-1]:
    Entropies.append(calculate_entropy(data,i))

# Return in format: ( attribute with minimum entropy and its entropy )
  return attr[np.argmin(np.array(Entropies))],min(Entropies)


In [ ]:
a='a'
if isinstance(a, list):
    print("yes")
else:
    print("no")


no
